In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

asv_df = pd.read_csv("/content/drive/MyDrive/VoiceSpoof/ASVspoof2021_LA_eval/Dataset/asv_key.csv")
thai_df = pd.read_csv("/content/drive/MyDrive/VoiceSpoof/ASVspoof2021_LA_eval/Dataset/thai_key.csv")


In [ ]:
# split

eval = asv_df[asv_df["7"] == 'eval'] # test
hidden = asv_df[asv_df["7"] == 'hidden'] # eval
progress = asv_df[asv_df["7"] == 'progress'] # train

thai_eval = thai_df[thai_df["set"] == "eval"] #thai train
thai_train = thai_df[thai_df["set"] == "train"] #thai eval

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

def stack_loader(X, y):
  X_tensor = torch.tensor(X, dtype=torch.float32)
  y_tensor = torch.tensor(y)
  dataset = TensorDataset(X_tensor, y_tensor)
  data_loader = DataLoader(dataset, batch_size=16, shuffle=True)
  return data_loader

In [ ]:
eval.head()

,file_name,0,2,3,4,labels,6,7,file_path
0,LA_E_6869886.flac,LA_0013,alaw,ita_tx,A07,0,notrim,eval,/content/drive/MyDrive/Voice Spoof/ASVspoof202...
1,LA_E_9900388.flac,LA_0039,opus,loc_tx,bonafide,1,notrim,eval,/content/drive/MyDrive/Voice Spoof/ASVspoof202...
3,LA_E_3024754.flac,LA_0043,gsm,sin_tx,A14,0,notrim,eval,/content/drive/MyDrive/Voice Spoof/ASVspoof202...
4,LA_E_3611176.flac,LA_0006,opus,sin_tx,A16,0,notrim,eval,/content/drive/MyDrive/Voice Spoof/ASVspoof202...
5,LA_E_8364839.flac,LA_0031,pstn,mad_tx,A11,0,notrim,eval,/content/drive/MyDrive/Voice Spoof/ASVspoof202...


# **Eng**

In [ ]:
import numpy as np
# Eng

progress_Raw = np.load("/content/drive/MyDrive/VoiceSpoof/ASVspoof2021_LA_eval/Features/4sec/Raw/Raw_progress.npy", allow_pickle=True)
hidden_Raw = np.load("/content/drive/MyDrive/VoiceSpoof/ASVspoof2021_LA_eval/Features/4sec/Raw/Raw_hidden.npy", allow_pickle=True)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/VoiceSpoof/ASVspoof2021_LA_eval/Features/4sec/Raw/Raw_progress.npy'

In [ ]:
#Eng
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_val = np.vstack([progress_Raw, hidden_Raw])
y_train = np.hstack([progress["labels"], hidden["labels"]])

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train, test_size=0.2, random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
train_loader = stack_loader(X_train_smote, y_train_smote)
val_loader = stack_loader(X_val, y_val)

In [ ]:
print("Shape of X_train:", X_train.shape) # progress 5055, hidden 5244
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (8239, 64000)
Shape of X_val: (2060, 64000)
Shape of y_train: (8239,)
Shape of y_val: (2060,)


# **Thai**

In [ ]:
import numpy as np
# Thai

thai_Raw = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Raw/Raw_thai_train.npy", allow_pickle=True)


In [ ]:
#Thai
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train, X_val, y_train, y_val = train_test_split(thai_Raw, thai_train["labels"], test_size=0.2, random_state=42)


In [ ]:
X_train.shape

(9600, 64000)

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

def stack_loader(X, y):
  X_tensor = torch.tensor(X, dtype=torch.float32)
  y_tensor = torch.tensor(y.values)
  dataset = TensorDataset(X_tensor, y_tensor)
  data_loader = DataLoader(dataset, batch_size=16, shuffle=True)
  return data_loader

train_loader = stack_loader(X_train, y_train)
val_loader = stack_loader(X_val, y_val)

In [ ]:
print("Shape of X_train:", X_train.shape) # thai real 1000, thai fake 11000
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (9600, 64000)
Shape of X_val: (2400, 64000)
Shape of y_train: (9600,)
Shape of y_val: (2400,)


# **Thai+Eng**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
# Eng

progress_Raw = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Raw/Raw_progress.npy", allow_pickle=True)
hidden_Raw = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Raw/Raw_hidden.npy", allow_pickle=True)

# Thai

thai_Raw = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Raw/Raw_thai_train.npy", allow_pickle=True)

X_train_val = np.vstack([thai_Raw, progress_Raw, hidden_Raw])
y_train = np.hstack([thai_train["labels"], progress["labels"], hidden["labels"]])

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train, test_size=0.2, random_state=42)

In [ ]:
def stack_loader(X, y):
  X_tensor = torch.tensor(X, dtype=torch.float32)
  y_tensor = torch.tensor(y)
  dataset = TensorDataset(X_tensor, y_tensor)
  data_loader = DataLoader(dataset, batch_size=16, shuffle=True)
  return data_loader

train_loader = stack_loader(X_train, y_train)
val_loader = stack_loader(X_val, y_val)

In [ ]:
print("Shape of X_train:", X_train.shape) # thai real 1000, thai fake 11000, progress 5055, hidden 5244
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)

Shape of X_train: (17839, 64000)
Shape of X_val: (4460, 64000)
Shape of y_train: (17839,)
Shape of y_val: (4460,)


# **RawNet2**

In [ ]:
!ls "/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Model"

RawNet2_Raw_Eng_100epoch.pth  RawNet2_Raw_Thai_100epoch.pth  RawNet2_Raw_Thai_Eng_100epoch.pth


In [ ]:
!git clone "https://github.com/asvspoof-challenge/2021.git"

Cloning into '2021'...
remote: Enumerating objects: 483, done.
remote: Counting objects: 100% (483/483), done.
remote: Compressing objects: 100% (309/309), done.
remote: Total 483 (delta 167), reused 450 (delta 135), pack-reused 0
Receiving objects: 100% (483/483), 2.30 MiB | 6.64 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [ ]:
%cd "/content/2021/LA/Baseline-RawNet2"

/content/2021/LA/Baseline-RawNet2


In [ ]:
import argparse
import sys
import os
import numpy as np
import torch
from torch import nn
from torch import Tensor
from torch.utils.data import DataLoader
import yaml
from data_utils import genSpoof_list,Dataset_ASVspoof2019_train,Dataset_ASVspoof2021_eval
from model import RawNet
# from tensorboardX import SummaryWriter
from core_scripts.startup_config import set_random_seed
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d

def evaluate_eer(dev_loader, model, device):
    model.eval()
    all_scores = []
    all_labels = []
    for batch_x, batch_y in dev_loader:
        batch_x = batch_x.to(device)
        batch_out = model(batch_x)
        scores = torch.softmax(batch_out, dim=1)[:, 1].cpu().detach().numpy()  # Get the probabilities for class 1
        labels = batch_y.cpu().detach().numpy()
        all_scores.extend(scores)
        all_labels.extend(labels)

    fpr, tpr, thresholds = roc_curve(all_labels, all_scores, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer

def evaluate_accuracy(dev_loader, model, device):
    num_correct = 0.0
    num_total = 0.0
    model.eval()
    for batch_x, batch_y in dev_loader:

        batch_size = batch_x.size(0)
        num_total += batch_size
        batch_x = batch_x.to(device)
        batch_y = batch_y.view(-1).type(torch.int64).to(device)
        batch_out = model(batch_x)
        _, batch_pred = batch_out.max(dim=1)
        num_correct += (batch_pred == batch_y).sum(dim=0).item()
    return 100 * (num_correct / num_total)

def train_epoch(train_loader, model, lr,optim, device):
    running_loss = 0
    num_correct = 0.0
    num_total = 0.0
    ii = 0
    model.train()

    #set objective (Loss) functions
    weight = torch.FloatTensor([0.1, 0.9]).to(device)
    criterion = nn.CrossEntropyLoss(weight=weight)

    for batch_x, batch_y in train_loader:

        batch_size = batch_x.size(0)
        num_total += batch_size
        ii += 1
        batch_x = batch_x.to(device)
        batch_y = batch_y.view(-1).type(torch.int64).to(device)
        batch_out = model(batch_x)
        batch_loss = criterion(batch_out, batch_y)
        _, batch_pred = batch_out.max(dim=1)
        num_correct += (batch_pred == batch_y).sum(dim=0).item()
        running_loss += (batch_loss.item() * batch_size)
        if ii % 10 == 0:
            sys.stdout.write('\r \t {:.2f}'.format(
                (num_correct/num_total)*100))
        optim.zero_grad()
        batch_loss.backward()
        optim.step()

    running_loss /= num_total
    train_accuracy = (num_correct/num_total)*100
    return running_loss, train_accuracy


In [ ]:
if __name__ == '__main__':

    model_save_path = "/content/drive/MyDrive/VoiceSpoof/ASVspoof2021_LA_eval/Model/Raw_smote"
    dir_yaml = os.path.splitext('model_config_RawNet')[0] + '.yaml'

    with open(dir_yaml, 'r') as f_yaml:
        parser1 = yaml.load(f_yaml, Loader=yaml.Loader)

    #GPU device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print('Device: {}'.format(device))

    #make experiment reproducible
    set_random_seed(1234)

    #model
    model = RawNet(parser1['model'], device)
    nb_params = sum([param.view(-1).size()[0] for param in model.parameters()])
    model =(model).to(device)

    #set Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.0001)

    num_epochs = 100
    best_acc = 99
    model_format = 'RawNet2_Raw_Eng_epoch_{}.pth'
    for epoch in range(num_epochs):
        running_loss, train_accuracy = train_epoch(train_loader,model, 0.0001, optimizer, device)
        valid_accuracy = evaluate_accuracy(val_loader, model, device)
        print('\n{} - {} - {:.2f} - {:.2f}'.format(epoch,
                                                   running_loss, train_accuracy, valid_accuracy))
        if valid_accuracy > best_acc:
            print('best model find at epoch', epoch)

        if epoch % 5 == 0 :
            best_acc = max(valid_accuracy, best_acc)
            torch.save(model.state_dict(), os.path.join(model_save_path, model_format.format(epoch)))

    torch.save(model.state_dict(), os.path.join(model_save_path, model_format.format(epoch)))

# **Test Model**

In [ ]:
import os
import numpy as np
import torch
from model import RawNet
from torch.utils.data import DataLoader, TensorDataset

# Assuming thaireal_Wav2vec and thaifake_Wav2vec are defined elsewhere

model_save_path = "/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Model"
dir_yaml = os.path.splitext('model_config_RawNet')[0] + '.yaml'

# Load model configuration from YAML file
with open(dir_yaml, 'r') as f_yaml:
    parser1 = yaml.load(f_yaml, Loader=yaml.Loader)

# GPU device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

model_path = os.path.join(model_save_path, "RawNet2_Raw_Thai_Eng_100epoch.pth")

# Create an instance of the model
model = RawNet(parser1['model'], device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()


Device: cpu


RawNet(
  (Sinc_conv): SincConv()
  (first_bn): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (selu): SELU(inplace=True)
  (block0): Sequential(
    (0): Residual_block(
      (lrelu): LeakyReLU(negative_slope=0.3)
      (conv1): Conv1d(20, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(20, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (mp): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (block1): Sequential(
    (0): Residual_block(
      (bn1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (lrelu): LeakyReLU(negative_slope=0.3)
      (conv1): Conv1d(20, 20, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(20, 20, kernel_size=(3,), stride

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
!ls "/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Raw/"

Raw_eval.npy  Raw_hidden.npy  Raw_progress.npy	Raw_thai_eval.npy  Raw_thai_train.npy


In [ ]:
import numpy as np
eval_raw = np.load("/content/drive/MyDrive/Voice Spoof/ASVspoof2021_LA_eval/Features/4sec/Raw/Raw_eval.npy")

y_test = eval["labels"]

In [ ]:
import pandas as pd

# Assuming 'labels' is your Pandas Series
y_test.reset_index(drop=True, inplace=True)

In [ ]:
print(y_test.index[~y_test.index.isin(range(len(y_test)))])

Int64Index([], dtype='int64')


In [ ]:
y_test.shape

(45952,)

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

def stack_loader(X, y):
  X_tensor = torch.tensor(X, dtype=torch.float32)
  y_tensor = torch.tensor(y)
  dataset = TensorDataset(X_tensor, y_tensor)
  data_loader = DataLoader(dataset, batch_size=16, shuffle=True)
  return data_loader

test_loader = stack_loader(eval_raw, y_test)

test_accuracy = evaluate_accuracy(test_loader, model, device)
test_eer = evaluate_eer(test_loader, model, device)

In [ ]:
eval_raw.shape

(45952, 64000)

In [ ]:
y_test.shape

(45952,)

In [ ]:
print("eval accuracy: ", test_accuracy)
print("eval err: ", test_eer)

eval accuracy:  93.68036211699165
eval accuracy:  0.11750494179661666


In [ ]:
# thai_eng 93.68036211699165, 0.11750494179661666,